In [2]:
import numpy as np
import tensorflow as tf
from PIL import Image
import matplotlib.pyplot as plt
import os
from IPython.display import display, clear_output
import ipywidgets as widgets
import io

# Load the trained model
model = tf.keras.models.load_model(r"C:\Users\nikhi\Desktop\archive\lung_cancer_model_fold_1.h5")

def get_class_name(class_index):
    class_names = {
        0: "Adenocarcinoma",
        1: "Large Cell Carcinoma",
        2: "Normal",
        3: "Squamous Cell Carcinoma"
    }
    return class_names.get(class_index, "Unknown")

def predict_image(img):
    try:
        # Resize and preprocess the image
        img = img.resize((224, 224))
        img_array = np.array(img)
        
        # Handle grayscale and RGBA images
        if len(img_array.shape) == 2:
            img_array = np.stack((img_array,)*3, axis=-1)
        elif img_array.shape[2] == 4:
            img_array = img_array[:,:,:3]
        
        # Preprocess the image
        img_array = tf.cast(img_array, tf.float32) / 255.0
        img_array = tf.image.per_image_standardization(img_array)
        img_array = np.expand_dims(img_array, axis=0)
        
        # Make prediction
        prediction = model.predict(img_array, verbose=0)
        predicted_class = np.argmax(prediction[0])
        confidence = prediction[0][predicted_class] * 100
        
        return prediction[0], predicted_class, confidence
        
    except Exception as e:
        print(f"Error processing image: {str(e)}")
        return None, None, None

def create_output_widgets():
    image_output = widgets.Output()
    text_output = widgets.Output()
    return image_output, text_output

def display_results(img, probabilities, predicted_class, confidence, image_output, text_output):
    with image_output:
        clear_output(wait=True)
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
        
        # Plot the image
        ax1.imshow(img)
        ax1.axis('off')
        ax1.set_title('Uploaded Image')
        
        # Plot probability distribution
        class_names = [get_class_name(i) for i in range(4)]
        colors = ['blue' if i != predicted_class else 'red' for i in range(4)]
        ax2.bar(class_names, probabilities * 100, color=colors)
        ax2.set_title('Prediction Probabilities')
        ax2.set_ylabel('Probability (%)')
        plt.xticks(rotation=45)
        
        plt.tight_layout()
        plt.show()
    
    with text_output:
        clear_output(wait=True)
        print(f"\nPredicted Class: {get_class_name(predicted_class)}")
        print(f"Confidence: {confidence:.2f}%")
        print("\nDetailed Probabilities:")
        for i in range(len(probabilities)):
            print(f"{get_class_name(i)}: {probabilities[i]*100:.2f}%")

def create_interactive_widget():
    # Create widgets
    upload_button = widgets.FileUpload(
        accept='.jpg,.jpeg,.png,.gif,.bmp',
        multiple=False,
        description='Select Image',
        button_style='info',
        icon='upload'
    )
    
    reset_button = widgets.Button(
        description='Reset',
        button_style='warning',
        icon='refresh'
    )
    
    status_label = widgets.Label(value="Upload an image to begin...")
    
    image_output, text_output = create_output_widgets()
    
    # Layout
    button_box = widgets.HBox([upload_button, reset_button])
    main_box = widgets.VBox([
        status_label,
        button_box,
        image_output,
        text_output
    ])
    
    def on_upload_change(change):
        if change['type'] == 'change' and change['name'] == 'value':
            # Get the uploaded file
            uploaded_files = change['new']
            if not uploaded_files:
                return
            
            status_label.value = "Processing image..."
            
            # Get the first file from the dictionary
            first_file_key = next(iter(uploaded_files))
            content = uploaded_files[first_file_key]['content']
            
            try:
                img = Image.open(io.BytesIO(content))
                
                # Make prediction
                probabilities, predicted_class, confidence = predict_image(img)
                
                if predicted_class is not None:
                    status_label.value = "Analysis complete!"
                    display_results(img, probabilities, predicted_class, confidence, 
                                 image_output, text_output)
                else:
                    status_label.value = "Error processing image. Please try again."
            except Exception as e:
                status_label.value = f"Error: {str(e)}"
                with text_output:
                    clear_output(wait=True)
                    print(f"Error processing image: {str(e)}")
    
    def on_reset_click(b):
        with image_output:
            clear_output()
        with text_output:
            clear_output()
        upload_button.value.clear()
        status_label.value = "Upload an image to begin..."
    
    # Register callbacks
    upload_button.observe(on_upload_change)
    reset_button.on_click(on_reset_click)
    
    return main_box

# Style settings
style = """
    <style>
        .widget-label { font-size: 16px; font-weight: bold; }
        .widget-upload { width: 200px; }
        .widget-button { width: 100px; }
    </style>
"""
display(widgets.HTML(style))

# Create and display the interactive widget
print("Lung Cancer Classification System")
print("--------------------------------")
interactive_widget = create_interactive_widget()
display(interactive_widget)

HTML(value='\n    <style>\n        .widget-label { font-size: 16px; font-weight: bold; }\n        .widget-uplo…

Lung Cancer Classification System
--------------------------------
